In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from darts import TimeSeries

df = pd.read_csv('../csv/Ming25Jun67_total_order.csv')
df.tail()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['order_completed_at'] = pd.to_datetime(df['order_completed_at'])

In [ ]:
# make timeseries Object
series = TimeSeries.from_dataframe(df,
                                   time_col='order_completed_at',
                                   value_cols=['unique_order_count'],
                                   freq="D")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.figure(figsize=(20, 10)) 
series.plot()

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gcf().autofmt_xdate() 

plt.title('Total Order By Date from 2022-2024')
plt.xlabel('Date')
plt.ylabel('Number of Orders (Unit)')

plt.show()

# Feature Enigineering

In [ ]:
# Function to replace outliers with the mean of the rest of the values
from scipy.stats import zscore
from darts import TimeSeries

def replace_outliers_with_mean(df, column_name):
    # Calculate Z-scores
    z_scores = zscore(df[column_name])
    
    # Identify outliers (using a threshold of 3 for Z-score)
    outliers = np.abs(z_scores) > 3
    
    # Calculate mean of non-outliers
    mean_non_outliers = df.loc[~outliers, column_name].mean()
    
    # Replace outliers with the mean of non-outliers
    df.loc[outliers, column_name] = mean_non_outliers
    
    return df

# Replace outliers in 'gmv' column
df_remove_out = replace_outliers_with_mean(df, 'unique_order_count')
series_remove_outlier = TimeSeries.from_dataframe(df_remove_out,
                                   time_col='order_completed_at',
                                   value_cols=['unique_order_count'],
                                   freq="D")
series = series_remove_outlier #!!!

In [ ]:
from darts.dataprocessing.transformers import (
    Scaler
)
scaler = Scaler()
series_rescaled = scaler.fit_transform(series)
series = series_rescaled

### split month

In [ ]:
# training, validation =series.split_before(pd.Timestamp('2024-01-01'))
# cutoff_date = pd.Timestamp('2024-05-01')
cut_off = 0.8
# training, validation =series.split_before(cutoff_date) # 1 month
training, validation =series.split_before(cut_off)
#training, validation =series.split_before(pd.Timestamp('2024-04-01')) # 2 month

In [ ]:
#check date split
validation_df = validation.pd_dataframe()
training_df = training.pd_dataframe()
print(training_df.tail())
print(validation_df.head())

In [ ]:
import matplotlib.dates as mdates

plt.figure(figsize=(20, 10))  # ปรับขนาดของภาพ (กว้าง x สูง)
training.plot()
validation.plot()

# ตั้งค่ารูปแบบของวันที่บนแกน x
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gcf().autofmt_xdate()  # ปรับให้วันที่ไม่ซ้อนกัน

# กำหนด Title และชื่อของแกน x และ y
plt.title('Total Order By Date from 2022-2024')
plt.xlabel('Date')
plt.ylabel('Number of Orders (Unit)')

plt.show()

In [ ]:
from darts.models import LightGBMModel

model = LightGBMModel(lags=24, output_chunk_length=7 ,random_state=42)
model.fit(training)
forecast = model.predict(len(validation))

In [ ]:
# plot Actual and Predict
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))  
series.plot(label="Actual")
forecast.plot(label="forecast", low_quantile=0.05, high_quantile=0.95)
plt.legend()

In [ ]:
from darts.metrics import mape, mae, mse, mase
from math import sqrt
#not recommendation

mape_score = mape(validation, forecast)
print(f'score MAPE: {mape_score}')

mase_score = mase(validation, forecast, training)
print(f'score MASE: {mase_score}')

In [ ]:
# historical forecast not found, only backtesting

backtest_errors = model.backtest(
     series,
     start=0.9,          # เริ่มต้นที่ 80% ของข้อมูล
     forecast_horizon=3, #จำนวนก้าวการพยากรณ์ในอนาคตที่ต้องการ
     stride=1,           #ช่วงการเลื่อนในการทดสอบแต่ละครั้ง
     last_points_only=True,  #  ให้แสดงเฉพาะจุดสิ้นสุดของการพยากรณ์หรือไม่ , เพราะต้องการแค่ทำนายวันพรุ่งนี้
     metric=mape,
     reduction=np.mean           #ฟังก์ชันการลดรูปผลลัพธ์ (เช่น mean การหาเฉลี่ย)
)

In [ ]:
print(f'backtest_errors MAPE: {round(backtest_errors,2)}%')

### gridsearch

In [ ]:
from darts.models import LightGBMModel
from darts.metrics import mape, mae, mse, mase
from math import sqrt

parameters = {
    "lags":list(range(1,36)),
    "output_chunk_length":  [7],  # ทดลอง output_chunk_length ที่แตกต่างกัน
    "random_state": [42]
}

best_parameter = LightGBMModel.gridsearch(
    parameters=parameters,
    series = series, #note (training + validation) X only training set!!!
    start=cut_off,
    forecast_horizon=7,
    stride=1,          
    last_points_only=False,  
    metric=mse,
    reduction=np.mean, 
    verbose=-1 
)

In [ ]:
print(best_parameter)

In [ ]:
from darts.models import LightGBMModel
from darts.metrics import mape, mae, mse, mase
from math import sqrt

# best_model = best_parameter[0]
best_model = best_parameter[0]
best_model.fit(training)

In [ ]:
prediction = best_model.predict(len(validation))

mape_score = mape(validation, prediction)
print(f'score MAPE: {round(mape_score, 4)}')
print(f'score ACC: {100-(round(mape_score, 4))}')

mse_score = mse(validation, prediction)
print(f'score MSE: {round(mse_score, 4)}')

print(f'score RMSE: {round(sqrt(mse_score), 4)}')

mae_score = mae(validation, prediction)
print(f'score MAE: {round(mae_score, 4)}')

In [ ]:
validation_rescale = scaler.inverse_transform(validation)
prediction_rescale = scaler.inverse_transform(prediction)

mape_score = mape(validation_rescale, prediction_rescale)
print(f'score MAPE: {round(mape_score, 4)}')
print(f'score ACC: {100-(round(mape_score, 4))}')

mse_score = mse(validation_rescale, prediction_rescale)
print(f'score MSE: {round(mse_score, 4)}')

print(f'score RMSE: {round(sqrt(mse_score), 4)}')

mae_score = mae(validation_rescale, prediction_rescale)
print(f'score MAE: {round(mae_score, 4)}')

In [ ]:
from darts.metrics import mape, mae, mse, mase
from math import sqrt
out_lst = []


backtest_errors = best_model.backtest(
     series,
     start=0.8,          # เริ่มต้นที่ 90% ของข้อมูล
     forecast_horizon=7, #จำนวนก้าวการพยากรณ์ในอนาคตที่ต้องการ
     stride=1,           #ช่วงการเลื่อนในการทดสอบแต่ละครั้ง
     last_points_only=False,  #  ให้แสดงเฉพาะจุดสิ้นสุดของการพยากรณ์หรือไม่
     metric=mape,
     reduction=np.mean           #ฟังก์ชันการลดรูปผลลัพธ์ (เช่น mean การหาเฉลี่ย)
)
print(f'backtest_errors MAPE: {backtest_errors}')

In [ ]:
# for out in out_lst:print(out)

In [ ]:
plt.figure(figsize=(20, 10))  # ปรับขนาดของภาพ (กว้าง x สูง)
##prediction
prediction.plot(label="prediction")
training.plot(label="training")
validation.plot(label="validation")

plt.legend()
plt.title('Train, Validation, and Prediction')
plt.xlabel('Date')
plt.ylabel('Order Unit')
plt.show()

In [ ]:
# predict future
best_model.fit(series)

In [ ]:
#check date
series.pd_dataframe().tail(14)

In [ ]:
forecast = best_model.predict(7)
forecast  = scaler.inverse_transform(forecast)
forecast = forecast.pd_dataframe().reset_index()
forecast.columns = ['Date', 'Total Order']
print(forecast.tail(7))

forecast.tail(7).to_csv('../forecast/25-06-67_lightGBM.csv', index=False)

### Save model

In [ ]:
best_model.save("../model/LightGBMModel_totalOrder_25Jun67_1lags.pkl")

In [ ]:
from darts.metrics import mape, mae, mse, mase
from math import sqrt
from darts.models import LightGBMModel

training, validation =series.split_before(pd.Timestamp('2024-05-01')) # 1 month

best_model = LightGBMModel.load("../model/LightGBMModel_totalOrder_25Jun67_1lags.pkl")
backtest_errors = best_model.backtest(
     series,
     start=pd.Timestamp('2024-05-01'),          # เริ่มต้นที่ 90% ของข้อมูล
     forecast_horizon=7, #จำนวนก้าวการพยากรณ์ในอนาคตที่ต้องการ
     stride=1,           #ช่วงการเลื่อนในการทดสอบแต่ละครั้ง
     last_points_only=False,  #  ให้แสดงเฉพาะจุดสิ้นสุดของการพยากรณ์หรือไม่
     metric=mse,
     reduction=np.mean           #ฟังก์ชันการลดรูปผลลัพธ์ (เช่น mean การหาเฉลี่ย)
)
print(f'backtest_errors MSE: {backtest_errors}')